In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install faiss-gpu

from transformers import AutoTokenizer, AutoModel
from datasets import load_dataset
from datasets import Dataset
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.8 MB/s eta 0:00:00


In [2]:
model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

In [3]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [4]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    #encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [ ]:
# import pickle

# embeddings_dataset1 = pickle.load(open('MultiQA_DocumentLevel_pkl','rb'))
# embeddings_dataset1

Dataset({
    features: ['Title', 'Subheading', 'Paragraph', 'URL', 'text', 'embeddings'],
    num_rows: 457
})

In [5]:
import pickle

embeddings_dataset2 = pickle.load(open('MultiQA_DocumentLevel_pkl','rb'))
embeddings_dataset2

Dataset({
    features: ['DocID', 'Title', 'URL', 'Embeddings'],
    num_rows: 69
})

In [6]:
question = ["Therapy for ADHD", "Not able to focus","Child not sleeping properly","bed wetting"]
question_embedding = get_embeddings([question[-1]]).cpu().detach().numpy()
#question_embedding

In [7]:
scores, samples = embeddings_dataset2.get_nearest_examples(
    "Embeddings", question_embedding, k=5
)

In [8]:
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

In [9]:
for _, row in samples_df.iterrows():
    print(f"TITLE: {row.Title}")
    print(f"SCORE: {row.scores}")
    # print(f"PARAGRAPH: {row.Paragraph}")
    print(f"URL: {row.URL}")
    print("=" * 50)
    print()

TITLE: Nightmares in Children | A Guide for Parents - ChildrensMD
SCORE: 45.50067138671875
URL: https://childrensmd.org/browse-by-topic/behavior-development/nightmares-in-children

TITLE: Why I Don't Co-Sleep - Motherly
SCORE: 43.427547454833984
URL: https://www.mother.ly/life/motherly-stories/kids-sleep-own-bed

TITLE: 6 Solutions to Your Child's Biggest Potty Training Problems
SCORE: 42.709381103515625
URL: https://www.parents.com/toddlers-preschoolers/potty-training/problems/solutions-to-your-childs-biggest-potty-training

TITLE: Potty Training: What to Know
SCORE: 42.43050765991211
URL: https://www.webmd.com/parenting/baby/potty-training-when-to-start#

TITLE: Page not found - Parenting
SCORE: 42.116390228271484
URL: https://www.parenting.com/article/how-to-help-kids-stop-wetting-the-be



Passage Level Semantic Search

In [ ]:
import pickle

embeddings_dataset2 = pickle.load(open('MultiQA_PassageLevel_pkl','rb'))
embeddings_dataset2

In [ ]:
question = ["Therapy for ADHD", "Not able to focus","Child not sleeping properly","bed wetting"]
question_embedding = get_embeddings([question[-1]]).cpu().detach().numpy()
#question_embedding

In [ ]:
scores, samples = embeddings_dataset2.get_nearest_examples(
    "Embeddings", question_embedding, k=5
)

In [ ]:
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

In [ ]:
for _, row in samples_df.iterrows():
    print(f"TITLE: {row.Title}")
    print(f"SCORE: {row.scores}")
    # print(f"PARAGRAPH: {row.Paragraph}")
    print(f"URL: {row.URL}")
    print("=" * 50)
    print()